## Cross-Validation

In this notebook, we will estimate the generalization error of a machine learning model using Cross-Validation Schemes.

In [27]:
# import libraries

import numpy as np
import pandas as pd

from scipy.special import comb

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.model_selection import (
    KFold,
    LeaveOneOut,
    LeavePOut,
    RepeatedKFold,
    StratifiedKFold,
    cross_validate,
    train_test_split
)

import warnings
warnings.filterwarnings("ignore")

In [2]:
# load data

# scikit-learn dataset
# https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

# dataset information: UCI Machine Learning Repository
# https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)
    
# in short, classification problem, trying to predict whether the tumor
# is malignant or benign

# load dataset
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
X.shape

(569, 30)

In [6]:
# target percentage
y.value_counts()/len(y)

0    0.627417
1    0.372583
dtype: float64

In [7]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

## K-Fold Cross Validation

In [8]:
# set up the logistic regression 

logit = LogisticRegression(C=10, solver='liblinear', max_iter=10000, n_jobs=2)

# set up kfold
kf = KFold(n_splits=5, shuffle=True, random_state=4)

# estimate generalization error using cross_validate
clf = cross_validate(logit,
               X_train,
               y_train,
               scoring='accuracy',
               return_train_score=True, # to get the training score as well
               cv=kf) # k-fold

C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\si

In [9]:
# clf is basically a dictionary that returns the scores and other cross validation results
clf

{'fit_time': array([0.07117915, 0.01178169, 0.01429009, 0.01297975, 0.01082826]),
 'score_time': array([0.02949333, 0.00343776, 0.00274491, 0.00100207, 0.0040679 ]),
 'test_score': array([0.925     , 0.95      , 0.9625    , 0.96202532, 0.94936709]),
 'train_score': array([0.97169811, 0.96540881, 0.96855346, 0.96238245, 0.97178683])}

In [10]:
# different scores on test data for 5 folds
clf['test_score']

array([0.925     , 0.95      , 0.9625    , 0.96202532, 0.94936709])

In [13]:
print('mean train accuracy :{} +- {}'.format(np.mean(clf['train_score']), np.std(clf['train_score'])))
print('mean test accuracy :{} +- {}'.format(np.mean(clf['test_score']), np.std(clf['test_score'])))

mean train accuracy :0.9679659312710711 +- 0.003649325403713798
mean test accuracy :0.9497784810126582 +- 0.013608919570498668


## Repeated K-Fold Cross Validation

In [14]:
# set up the logistic regression 

logit = LogisticRegression(C=10, solver='liblinear', max_iter=10000, n_jobs=2)

# set up kfold
rkf = RepeatedKFold(n_splits=5,
                    n_repeats=10,
                    random_state=4)
# here we are expecting 5 * 10 = 50 results i.e. k * n performance metrics

# estimate generalization error using cross_validate
clf = cross_validate(logit,
               X_train,
               y_train,
               scoring='accuracy',
               return_train_score=True, # to get the training score as well
               cv=rkf) # k-fold

C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\si

C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
C:\Users\budha\AppData\Local\Programs\Python\Python39\lib\si

In [16]:
# there will be 50 performance metrics
len(clf['test_score'])

50

In [17]:
clf['test_score']

array([0.925     , 0.95      , 0.9625    , 0.96202532, 0.94936709,
       0.95      , 0.9625    , 0.925     , 0.98734177, 0.94936709,
       0.9875    , 0.9375    , 1.        , 0.91139241, 0.94936709,
       0.9375    , 0.9625    , 0.9625    , 0.92405063, 0.96202532,
       0.975     , 0.9375    , 0.9625    , 0.96202532, 0.96202532,
       0.9875    , 0.9125    , 0.9625    , 0.92405063, 0.94936709,
       0.95      , 0.95      , 0.95      , 0.92405063, 0.97468354,
       0.95      , 0.95      , 0.975     , 1.        , 0.89873418,
       0.925     , 0.9375    , 0.975     , 0.94936709, 0.97468354,
       0.925     , 0.9625    , 0.8875    , 0.93670886, 0.98734177])

In [18]:
print('mean train accuracy :{} +- {}'.format(np.mean(clf['train_score']), np.std(clf['train_score'])))
print('mean test accuracy :{} +- {}'.format(np.mean(clf['test_score']), np.std(clf['test_score'])))

mean train accuracy :0.9672745016856923 +- 0.0072961432233506094
mean test accuracy :0.9515094936708861 +- 0.024498213076043808


## Leave One Out

In [21]:
# set up the logistic regression 

logit = LogisticRegression(C=10, solver='liblinear', max_iter=10000, n_jobs=2)

# set up kfold
loo = LeaveOneOut()

print('We expect as many metrics as data in the train set: ', len(X_train))

# estimate generalization error using cross_validate
clf = cross_validate(logit,
               X_train,
               y_train,
               scoring='accuracy',
               return_train_score=True, # to get the training score as well
               cv=loo) # k-fold

print('Number of metrics obtained: ', len(clf['test_score']))

len(clf['test_score'])

We expect as many metrics as data in the train set:  398
Number of metrics obtained:  398


398

In [22]:
print('mean train accuracy :{} +- {}'.format(np.mean(clf['train_score']), np.std(clf['train_score'])))
print('mean test accuracy :{} +- {}'.format(np.mean(clf['test_score']), np.std(clf['test_score'])))

mean train accuracy :0.9652671417541105 +- 0.0033268451073502456
mean test accuracy :0.9522613065326633 +- 0.21321282938268124


## Leave P Out

In [28]:
# set up the logistic regression 

logit = LogisticRegression(C=10, solver='liblinear', max_iter=10000, n_jobs=2)

# set up kfold
lpo = LeavePOut(p=3)

# I take a smaller sample of the data, otherwise
# my computer runs out of memory
X_train_small = X_train.head(100)
y_train_small = y_train.head(100)

# The number of combinations of N things taken k at a time.
print('We expect : ', comb(100,2), ' metrics')

# estimate generalization error using cross_validate
clf = cross_validate(logit,
               X_train_small,
               y_train_small,
               scoring='accuracy',
               return_train_score=True, # to get the training score as well
               cv=lpo) # leave p out

print('Number of metrics obtained: ', len(clf['test_score']))

len(clf['test_score'])

We expect :  4950.0  metrics
Number of metrics obtained:  161700


161700

In [29]:
print('mean train accuracy :{} +- {}'.format(np.mean(clf['train_score']), np.std(clf['train_score'])))
print('mean test accuracy :{} +- {}'.format(np.mean(clf['test_score']), np.std(clf['test_score'])))

mean train accuracy :0.9829649535540551 +- 0.005639951675759749
mean test accuracy :0.9489342403628119 +- 0.12846448710196262


## Stratified K-Fold Cross-Validation

In [23]:
# set up the logistic regression 

logit = LogisticRegression(C=10, solver='liblinear', max_iter=10000, n_jobs=2)

# set up kfold
sko = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)

# estimate generalization error using cross_validate
clf = cross_validate(logit,
               X_train,
               y_train,
               scoring='accuracy',
               return_train_score=True, # to get the training score as well
               cv=sko) # k-fold

print('Number of metrics obtained: ', len(clf['test_score']))

len(clf['test_score'])

Number of metrics obtained:  5


5

In [24]:
clf['test_score']

array([0.975     , 0.95      , 0.9625    , 0.94936709, 0.91139241])

In [25]:
print('mean train accuracy :{} +- {}'.format(np.mean(clf['train_score']), np.std(clf['train_score'])))
print('mean test accuracy :{} +- {}'.format(np.mean(clf['test_score']), np.std(clf['test_score'])))

mean train accuracy :0.9698487805839789 +- 0.0042678984389350584
mean test accuracy :0.9496518987341773 +- 0.02131474180547955
